# Mushroom Toxicity Classification Technical Report 


## Introduction

## Data Analysis

In [9]:
import importlib
import utils
import mypytable
importlib.reload(mypytable)
from mypytable import MyPyTable
import mysklearn.myknnclassifier as myknnclassifier, mynaivebayesclassifier
import myeval

mush_data = MyPyTable()
mush_data.load_from_file('/home/CPSC322finalProject/new_mushroom_cleaned.csv')

### KNN

In [10]:
# train test split and data set shrink
indexes_to_remove = utils.rand_inds(50000, 54034)
mush_data.drop_rows(indexes_to_remove)


#print(len(mush_data.data))

df_mush = []
for i, row in enumerate(mush_data.data):
    df_mush.append(row)

#print(len(df_mush))


season = [row[7] for row in df_mush]
stem_width = [row[5] for row in df_mush]
gill_color = [row[3] for row in df_mush]
cap_diameter = [row[0] for row in df_mush]

yummy_or_nah = [row[8] for row in df_mush]

# normalize the data
season_norm = utils.normalize_data(season)
stem_width_norm = utils.normalize_data(stem_width)
gill_color_norm = utils.normalize_data(gill_color)
cap_diameter_norm = utils.normalize_data(cap_diameter)

data_zipped = list(zip(season_norm, stem_width_norm, gill_color_norm, cap_diameter_norm))
X_data = data_zipped

X_train, X_test, y_train, y_test = myeval.train_test_split(X_data, yummy_or_nah, test_size=0.33, random_state=1, shuffle=True)
print(len(y_test))
print(len(X_test))
print(len(X_train))
print(len(y_train))

1332
1332
2703
2703


In [11]:
# initiate the knn
knn = myknnclassifier.MyKNeighborsClassifier(n_neighbors= 5)

knn.fit(X_train, y_train)

knn_y_pred = knn.predict(X_test)

In [12]:
acc = utils.compute_accuracy(knn_y_pred, y_test)
print(f'Accuacy: {acc:.4f}')

recall = utils.compute_recall(knn_y_pred, y_test, positive_label=1)
print(f'Recall: {recall:.4f}')

Accuacy: 0.8026
Recall: 0.8288


### Naive Bayes

In [13]:
y = [row[-1] for row in mush_data.data]   # Labels: survived
k = 10
# Stratified k-fold split
folds = myeval.stratified_kfold_split(X_data, y, n_splits=k, shuffle=True, random_state=2)


TypeError: randomize_in_place() got an unexpected keyword argument 'seed'